## Preambule

In [354]:
# General packages
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib as mpl
from pathlib import Path
import xarray as xr
import json

# Plotting
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import n_colors

## Parameters

In [2]:
hist_startyear = 1990

## Paths

In [3]:
path_main = Path("X:/user/dekkerm/Projects/ECEMF_T5.2")
path_data = path_main / "Data" / str(hist_startyear)
path_figs = path_main / "Figures"
path_ar6_iso = Path("X:/user/dekkerm/Data/IPCC/AR6_ISO3/")

## Read data files

In [4]:
xr_total = xr.open_dataset(path_data / "xr_total.nc")
all_regions_iso = np.load(path_data / "all_regions.npy")
all_regions_names = np.load(path_data / "all_regions_names.npy")
all_countries_iso = np.load(path_data / "all_countries.npy", allow_pickle=True)
all_countries_names = np.load(path_data / "all_countries_names.npy", allow_pickle=True)
only_regions_iso = ['CVF', 'G20', "EU", "G7", "NA", "AU", "AF", "SIDS", "LDC", "WORLD"]
only_regions_names = ['Climate Vulnerability Forum','G20',"European Union","G7","Northern America","Australasia","African Group","Small Island Developing States","Least Developed Countries"]

In [5]:
xr_budgets_static = xr.open_dataset(path_data / "xr_budgets_static.nc")
xr_budgets_linear = xr.open_dataset(path_data / "xr_budgets_linear.nc")
xr_budgets_scenario = xr.open_dataset(path_data / "xr_budgets_scenario.nc")
df_michel = pd.read_excel(Path("X:/user/dekkerm/Data/") / "NDC" / "NDC_comparison.xlsx", sheet_name="Michel")

In [6]:
df_countries = pd.read_csv("X:/user/dekkerm/Data/AllCountries.csv")

In [7]:
df_data = pd.read_csv(path_ar6_iso / "AR6_Scenarios_Database_ISO3_v1.1.csv")

In [8]:
df_meta = pd.read_excel(path_ar6_iso / "AR6_Scenarios_Database_metadata_indicators_v1.1.xlsx", sheet_name= "meta_Ch3vetted_withclimate")

## Preprocess

In [273]:
mods = np.array(df_meta[(df_meta.Category == "C1") | (df_meta.Category == "C2")].Model)
scens = np.array(df_meta[(df_meta.Category == "C1") | (df_meta.Category == "C2")].Scenario)
modscens = [mods[i]+'|'+scens[i] for i in range(len(mods))]
mods = np.array(df_data.Model)
scens = np.array(df_data.Scenario)
modscens_data = [mods[i]+'|'+scens[i] for i in range(len(scens))]
modscens_data_uni = np.unique([mods[i]+'|'+scens[i] for i in range(len(scens))])
df_data['ModelScenario'] = modscens_data
df_data_small = df_data[df_data.ModelScenario.isin(modscens)]
df_data_small = df_data_small.reset_index(drop=True)
df_data_small = df_data_small[["ModelScenario", "Region", "Variable"]+['1990', '1995',
       '2000', '2005', '2007', '2008', '2010', '2011', '2012', '2013', '2014',
       '2015', '2018', '2020', '2021', '2022', '2023', '2024', '2025', '2030',
       '2035', '2040', '2045', '2050', '2055', '2060', '2065', '2070', '2075',
       '2080', '2085', '2090', '2095', '2100']]
df__ = df_data_small.melt(id_vars=["ModelScenario", "Region", "Variable"], var_name="Time", value_name="Value")
df__['Time'] = df__['Time'].astype(int)
df__ = df__.set_index(['ModelScenario', 'Region', 'Variable', 'Time'])
xr_data_15 = xr.Dataset.from_dataframe(df__)

In [38]:
# mods = np.array(df_meta[(df_meta.Category == "C3")].Model)
# scens = np.array(df_meta[(df_meta.Category == "C3")].Scenario)
# modscens = [mods[i]+'|'+scens[i] for i in range(len(mods))]
# mods = np.array(df_data.Model)
# scens = np.array(df_data.Scenario)
# modscens_data = [mods[i]+'|'+scens[i] for i in range(len(scens))]
# modscens_data_uni = np.unique([mods[i]+'|'+scens[i] for i in range(len(scens))])
# df_data['ModelScenario'] = modscens_data
# df_data_small = df_data[df_data.ModelScenario.isin(modscens)]
# df_data_small = df_data_small.reset_index(drop=True)
# df_data_small = df_data_small[["ModelScenario", "Region", "Variable"]+['1990', '1995',
#        '2000', '2005', '2007', '2008', '2010', '2011', '2012', '2013', '2014',
#        '2015', '2018', '2020', '2021', '2022', '2023', '2024', '2025', '2030',
#        '2035', '2040', '2045', '2050', '2055', '2060', '2065', '2070', '2075',
#        '2080', '2085', '2090', '2095', '2100']]
# df__ = df_data_small.melt(id_vars=["ModelScenario", "Region", "Variable"], var_name="Time", value_name="Value")
# df__['Time'] = df__['Time'].astype(int)
# df__ = df__.set_index(['ModelScenario', 'Region', 'Variable', 'Time'])
# xr_data_2 = xr.Dataset.from_dataframe(df__)

In [274]:
xr_data_15 = xr_data_15.reindex(Time = np.arange(1990, 2101))
xr_data_15 = xr_data_15.interpolate_na(dim="Time", method="linear")

x:\user\dekkerm\environments\MyEnv\lib\site-packages\xarray\core\missing.py:264: FutureWarning:

is_monotonic is deprecated and will be removed in a future version. Use is_monotonic_increasing instead.



In [40]:
# xr_data_2 = xr_data_2.reindex(Time = np.arange(1990, 2101))
# xr_data_2 = xr_data_2.interpolate_na(dim="Time", method="linear")

x:\user\dekkerm\environments\MyEnv\lib\site-packages\xarray\core\missing.py:264: FutureWarning:

is_monotonic is deprecated and will be removed in a future version. Use is_monotonic_increasing instead.



Get new data

In [275]:
regs = np.array(['ARG', 'AUS', 'BRA', 'CAN', 'CHN', 'COL', 'EU', 'IDN', 'IND', 'JPN', 'KOR', 'MEX', 'PAK', 'RUS', 'SAU', 'TUR', 'USA', 'ZAF'])#np.array(xr_data_15.Region) -> without Taiwan!

In [276]:
cost_optimal_15 = np.array(xr_data_15.sel(Time=np.arange(2020, 2101), Variable="Emissions|Kyoto Gases", Region=regs).Value.sum(dim='Time'))/1e3
cost_optimal_15[cost_optimal_15 == 0] = np.nan
#np.array(xr_data_15.sel(Time=2030, Variable="Emissions|Kyoto Gases", Region=regs).Value)/1e3#np.array((xr_data_15.sel(Time=2019, Variable="Emissions|Kyoto Gases", Region=regs).Value - xr_data_15.sel(Time=2030, Variable="Emissions|Kyoto Gases", Region=regs).Value) /xr_data_15.sel(Time=2019, Variable="Emissions|Kyoto Gases", Region=regs).Value)
#cost_optimal_2 =  np.array((xr_data_2.sel(Time=2019, Variable="Emissions|Kyoto Gases", Region=regs).Value - xr_data_2.sel(Time=2030, Variable="Emissions|Kyoto Gases", Region=regs).Value) /xr_data_2.sel(Time=2019, Variable="Emissions|Kyoto Gases", Region=regs).Value)

In [535]:
#xr2 = xr_budgets_scenario.sel(Time=2030)#(xr_budgets_scenario.sel(Time=2020) - xr_budgets_scenario.sel(Time=2030))/xr_budgets_scenario.sel(Time=2020)
xr2 = xr_budgets_static
alloc_gf = np.array(xr2.sel(Region=regs,Variable='GF', Ccat='C1+C2').Value)
alloc_pc = np.array(xr2.sel(Region=regs, Variable='PC', Ccat='C1+C2').Value)
alloc_pcc = np.array(xr2.sel(Region=regs, Variable='PCC', Ccat='C1+C2').Value)
alloc_ap = np.array(xr2.sel(Region=regs, Variable='AP', Ccat='C1+C2').Value)
alloc_gdr = np.array(xr2.sel(Region=regs, Variable='GDR', Ccat='C1+C2').Value)
alloc_ecpc = np.array(xr2.sel(Region=regs, Variable='ECPC', Ccat='C1+C2').Value)
alloc_base = np.array(xr_total.GHG_base.sel(ISO=regs, Time=np.arange(2020, 2101)).sum(dim='Time', skipna=True))
ndc_h = np.array(xr_total.NDC_h.sel(ISO=regs, Time=np.arange(2020, 2101)))
ndc_l = np.array(xr_total.NDC_l.sel(ISO=regs, Time=np.arange(2020, 2101)))
allocs = [alloc_gf, alloc_pc, alloc_pcc, alloc_ap, alloc_gdr, alloc_ecpc]

In [537]:
total = np.zeros(shape=(len(allocs), len(regs)))
for alloc_i, alloc in enumerate(allocs):
    wh = np.where(alloc < np.nanmedian(cost_optimal_15, axis=0))[0]
    total[alloc_i, wh] = 1
fractions = np.sum(total, axis=0) / len(allocs)

## Plot

In [538]:
path_ctygroups = "X:/user/dekkerm/Data/" + "UNFCCC_Parties_Groups_noeu.xlsx"
df = pd.read_excel(path_ctygroups, sheet_name = "Country groups")
countries_iso = np.array(df["Country ISO Code"])
countries_name = np.array(df["Name"])
countries_iso = np.array(df["Country ISO Code"])
group_eu = countries_iso[np.array(df["EU"]) == 1]

In [558]:
fig = make_subplots(rows=5, cols=4, #subplot_titles=('Grandfathering (GF)', 'Per Capita (PC)', 'Per Capita Convergence (PCC)', 'Bretschger (BR)', 'Ability to pay (AP)', 'Greenhouse Development Rights (GDR)'), 
                                                    specs=[[{'type': 'choropleth', 'colspan': 3, 'rowspan': 3}, {}, {}, {}],
                                                           [{}, {}, {}, {}],
                                                           [{}, {}, {}, {}],
                                                           [{}, {}, {}, {}],
                                                           [{'colspan': 4}, {}, {}, {}]],
                                                    horizontal_spacing = 0.05, vertical_spacing=0.05)

colorbar_abs = dict(len=1, x=1, tickvals = [0, 1, 2, 3], ticktext = ['1', '10', '100', '1000'])
colorbar_pc = dict(len=1, x=1)

colmap = 'Earth_r'
colmap2 = 'terrain_r'
t = fractions#np.nanmean(cost_optimal_15, axis=0)
fig.add_trace(
    go.Choropleth(
    locations=regs,
    z = t,
    locationmode = 'ISO-3',
    colorscale = colmap,
    zmax = 1,
    zmin = 0,
    text = [str(r)+": "+str(int(t[r_i]*5))+" out of 5" for r_i, r in enumerate(regs)],
    hovertemplate  = '%{text}',
    name="",
    marker_line_color='black', 
    marker_line_width=1.,
    colorbar=dict(len=0.4, x=0.37, y=0.45, orientation='h',
                  tickvals = np.arange(0, 1.1, 1/6),
                  thickness=15,
                  tickfont={'size': 10},
                  ticktext=['None<br>(Mitigation financed<br>by others)', '1 out of 6', '2 out of 6', '3 out of 6', '4 out of 6', '5 out of 6', 'All<br>(Finances mitigation<br>abroad)'],
                  ticklabelposition='outside',
    ), #title={"text":"Number of effort-sharing<br>rules allocating less than<br>cost-optimal"}),
    showscale=True
), 1, 1)
fig.add_trace(
    go.Choropleth(
    locations=group_eu,
    z = [t[np.where(regs == "EU")[0][0]]]*len(group_eu),
    locationmode = 'ISO-3',
    colorscale = colmap,
    zmax = 1,
    zmin = 0,
    text = ["EU: "+str(int(t[np.where(regs == "EU")[0][0]]*len(allocs)))+" out of 6"for r_i, r in enumerate(group_eu)],
    hovertemplate  = '%{text}',
    name="",
    marker_line_color='black', 
    marker_line_width=1.,
    showscale=False
), 1, 1)
fig.update_geos(showocean=True, oceancolor="aliceblue")
fig.update_geos(showlakes=True, lakecolor="aliceblue")

# limite map scope to USA 'africa', 'asia', 'europe', 'north america', 'south america', 'usa', 'world'
fig.update_geos(visible=True,
                showlakes=False,
                lakecolor='rgb(255, 255, 255)',
                projection_type='natural earth',
                showcoastlines=False,)

base = np.array(xr_total.NDC_l.sel(ISO=regs, Time=2019))
for cty_i, cty in enumerate(["USA", "BRA", "ZAF", "EU", "RUS", "CHN", "IND"]):
    row = [4, 4, 4, 1, 2, 3, 4][cty_i]
    col = [1, 2, 3, 4, 4, 4, 4][cty_i]
    #fig.add_trace(go.Scatter(x=['BAU'], y=[alloc_base[regs == cty][0]], mode='markers', marker={'symbol':'diamond', "color": 'grey', "size": 13, "line": {"width": 1.5, "color": 'black'}}, showlegend=False), row=row, col=col)
    #fig.add_trace(go.Scatter(x=["NDC"], y=[((base - ndc_h) / base)[regs == cty][0]], mode='markers', marker={'symbol': "square", "color": 'black', "size": 10, "line": {"width": 0.2, "color": 'black'}}, showlegend=False), row=row, col=col)
    vals = np.copy(cost_optimal_15)#(cost_optimal_15 - base) / base
    vals = vals[:, regs == cty][:, 0]
    fig.add_trace(go.Scatter(x=['CO', 'GF', 'PC', 'PCC', 'AP', 'GDR', 'ECPC'], y=[np.nanmedian(vals)]*7, name='', showlegend=False, hoverinfo='skip', mode='lines', line={'color': 'black', 'width': 1}), col=col, row=row)
    fig.add_trace(go.Scatter(x=['GF', 'PC', 'PCC', 'AP', 'GDR', 'ECPC'], y=[np.nanpercentile(vals, 25)]*6, fill=None, name='', showlegend=False, hoverinfo='skip', mode='lines', line={'color': 'silver', 'width': 0}), col=col, row=row)
    fig.add_trace(go.Scatter(x=['GF', 'PC', 'PCC', 'AP', 'GDR', 'ECPC'], y=[np.nanpercentile(vals, 75)]*6, fill='tonexty', name='', showlegend=False, hoverinfo='skip', mode='lines', line={'color': 'silver', 'width': 0}), col=col, row=row)
    fig.add_trace(
                        go.Box(y=vals,
                        name="CO",
                        text=[cty+": "+str(np.round(vals[i],1))+"<br>"+modscens[i] for i in range(len(vals))],
                        fillcolor=plotly.colors.diverging.Earth_r[int(fractions[regs == cty][0]*7)],
                        line_color='black', marker=dict(color='grey'),#plotly.colors.diverging.Earth_r[int(fractions[regs == cty][0]*7)]),
                        boxpoints='all', jitter=0.5, pointpos=-1.8, whiskerwidth=0.4, marker_size=3, line_width=1.5, showlegend=False, hovertemplate='%{text}'),
                        row=row, col=col)
            # fig.add_trace(go.Scatter(x=[reg], y=[(base - budget_pc) / base], text="PC", textfont = {'family': "Arial black", "size": 13, "color": "black"},  marker={'symbol': "square", "color": cols[reg_i], "size": 25, "line": {"width": 0.2, "color": 'black'}, "colorscale": "Spectral",}, mode='markers+text', showlegend=False), row=1, col=1)
            # fig.add_trace(go.Scatter(x=[reg], y=[(base - budget_pcc) / base], text="PCC", textfont = {'family': "Arial black", "size": 13, "color": "black"},  marker={'symbol': "diamond", "color": cols[reg_i], "size": 25, "line": {"width": 0.2, "color": 'black'}, "colorscale": "Spectral",}, mode='markers+text', showlegend=False), row=1, col=1)
            # fig.add_trace(go.Scatter(x=[reg], y=[(base - ndc) / base], text="NDC", textfont = {'family': "Arial black", "size": 13, "color": "black"},  marker={'symbol': "triangle-up", "color": cols[reg_i], "size": 25, "line": {"width": 0.2, "color": 'black'}, "colorscale": "Spectral",}, mode='markers+text', showlegend=False), row=1, col=1)

    for alloc_i, alloc in enumerate(['GF', 'PC', 'PCC', 'AP', 'GDR', 'ECPC']):
        ally = allocs[alloc_i][regs == cty][0]
        if ally < np.nanpercentile(vals, 25):
            colr = 'tomato'
            marker = 'triangle-down'
            textpos = 'middle left'
            txt = '-<b>$'
            size = 13
        elif ally > np.nanpercentile(vals, 75):
            colr = 'forestgreen'
            marker = 'triangle-up'
            textpos = 'middle left'
            txt = '+<b>$'
            size = 13
        else:
            colr = 'grey'
            marker = 'square'
            textpos = 'middle left'
            txt = '~<b>$'
            size = 10
        fig.add_trace(go.Scatter(x=[alloc, alloc], y=[np.nanmedian(vals), ally], mode='lines', line={'width': 2, 'color': colr, 'dash': 'dot'}, showlegend=False), row=row, col=col)
        fig.add_trace(go.Scatter(x=[alloc], y=[ally], text=txt, textfont={'color': colr}, mode='markers+text', marker={'symbol':marker, "color": colr, "size": size, "line": {"width": 1.5, "color": 'black'}}, showlegend=False, textposition=textpos), row=row, col=col)

vals = xr_data_15.sel(Variable='Policy Cost|Consumption Loss', Time=np.arange(2020, 2101)).sum(dim='Time', skipna=False).Value
vals = vals / xr_data_15.sel(Variable='GDP|PPP', Time=2019).Value

for reg_i, reg in enumerate(regs[np.array(np.argsort(vals.sel(Region=regs).median(dim='ModelScenario')))]):
    y = np.array(vals.sel(Region=reg))
    if np.nansum(y) != 0:
        fig.add_trace(go.Box(name=all_regions_names[all_regions_iso == reg][0].replace(' ', '<br>'), y=y,
                            line_color='black',
                            marker=dict(color='grey'),#plotly.colors.diverging.Earth_r[int(fractions[regs == reg][0]*7)]),#, line={'width': 1, 'color': 'black'}),
                            fillcolor=plotly.colors.diverging.Earth_r[int(fractions[regs == reg][0]*7)],
                            boxpoints='all', jitter=0.5, pointpos=-1.8, whiskerwidth=0.4, marker_size=3, line_width=1.5,
                            showlegend=False), row=5, col=1)
    
# Panel labels
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=18),
                   y=1.04, x=0.20, text="<b>(a) Fair-share allocations less than cost-optimal</b><br>          (in number of effort-sharing approaches)")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='right', ay=-0, showarrow=False, font=dict(color='black', size=16), y=0.38, x=0.01, text="<b>(b) United States of America")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='right', ay=-0, showarrow=False, font=dict(color='black', size=16), y=0.38, x=0.27, text="<b>(c) Brazil")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='right', ay=-0, showarrow=False, font=dict(color='black', size=16), y=0.38, x=0.60, text="<b>(d) South-Africa")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='right', ay=-0, showarrow=False, font=dict(color='black', size=16), y=1.02, x=0.94, text="<b>(e) European Union")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='right', ay=-0, showarrow=False, font=dict(color='black', size=16), y=0.81, x=0.87, text="<b>(f) Russia")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='right', ay=-0, showarrow=False, font=dict(color='black', size=16), y=0.58, x=0.87, text="<b>(g) China")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='right', ay=-0, showarrow=False, font=dict(color='black', size=16), y=0.38, x=0.87, text="<b>(h) India")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=16), y=0.14, x=0.01, text="<b>(i) Policy costs over 2020-2100 in cost-optimal scenarios</b>")

# ADD LINES TO PATHS
# args = dict(xref='paper', yref='paper', type="line")
# fig.add_shape(y0=0.97, y1=0.97,
#               x0=0.76, x1=0.39, **args, line=dict(color=plotly.colors.diverging.Earth_r[int(fractions[regs == "EU"][0]*7)], width=3))
# fig.add_shape(y0=0.97, y1=0.85,
#               x0=0.39, x1=0.39, **args, line=dict(color=plotly.colors.diverging.Earth_r[int(fractions[regs == "EU"][0]*7)], width=3))
# fig.add_shape(y0=0.76, y1=0.76,
#               x0=0.76, x1=0.39, **args, line=dict(color=plotly.colors.diverging.Earth_r[int(fractions[regs == "RUS"][0]*7)], width=3))
# fig.add_shape(y0=0.97, y1=0.85,
#               x0=0.39, x1=0.39, **args, line=dict(color=plotly.colors.diverging.Earth_r[int(fractions[regs == "RUS"][0]*7)], width=3))
fig.update_yaxes(title_text="Full-century GHG emission<br>budget [Gt CO2e]", row=4, col=1)
fig.update_yaxes(title_text="Cumulative consumption<br>loss as fraction<br>of GDP in 2019", row=5, col=1)

fig.update_yaxes(rangemode="tozero")
fig.update_xaxes(zeroline=True, zerolinewidth=2, zerolinecolor='Black')
fig.update_layout(height=1100,
                  width=1400,
                  template='simple_white')
fig.show()

In [518]:
# titles = ["GHG emission reduction in 2030", "Policy costs in 2030", "CCS in 2030", "Primary biomass traded in 2030"]
# fig = make_subplots(rows=3, cols=3, subplot_titles=titles,
#                     horizontal_spacing = 0.025,
#                     specs=[[{"colspan": 3,"rowspan": 2}, None, None], [None, None, None], [{},{}, {}]],
#                     column_widths=[0.33, 0.33, 0.33],
#                     vertical_spacing=0.05)

# cols = ['forestgreen', 'tomato', 'steelblue', 'goldenrod', 'purple', 'grey', 'brown',
#         'magenta', 'red', 'darkgrey', 'blue', 'darkgreen']
# for reg_i, reg in enumerate(['BRA', 'USA', 'CHN', 'IND', 'IDN', "RUS", "EU", "AUS", "JPN", "COL", "MEX", "SAU"]):
#     for v_i in range(4):
#         row = [1, 3, 3 ,3][v_i]
#         col = [1, 1, 2, 3][v_i]
#         if v_i == 1: vals = np.array(xr_data.sel(Time=np.arange(2020, 2101), Region=reg, Variable="Policy Cost|Consumption Loss").Value.sum(dim='Time'))
#         if v_i == 0: vals = np.array((xr_data.sel(Time=2019, Region=reg, Variable="Emissions|Kyoto Gases").Value - xr_data.sel(Time=2030, Region=reg, Variable="Emissions|Kyoto Gases").Value) /xr_data.sel(Time=2019, Region=reg, Variable="Emissions|Kyoto Gases").Value)
#         if v_i == 2: vals = np.array(xr_data.sel(Time=2030, Region=reg, Variable="Carbon Sequestration|CCS").Value)
#         if v_i == 3: vals = np.array(xr_data.sel(Time=2030, Region=reg, Variable="Trade|Primary Energy|Biomass|Volume").Value)
#         vals[vals == 0] = np.nan
#         fig.add_trace(
#                         go.Box(y=vals,
#                         name=reg,
#                         text=[reg+"<br>"+titles[v_i]+": "+str(np.round(vals[i],1))+"<br>"+modscens[i] for i in range(len(vals))],
#                         fillcolor=cols[reg_i],
#                         line_color='black', marker=dict(color='black'),
#                         boxpoints='all', jitter=0.5, pointpos=-1.8, whiskerwidth=0.2, marker_size=3, line_width=0.5, showlegend=False, hovertemplate='%{text}'),
#                         row=row, col=col)
#         if v_i == 0:
#             base = float(xr_total.NDC_l.sel(ISO=reg, Time=2019)) #np.nanmean(xr_data.sel(Time=2020, Region=reg, Variable="Emissions|Kyoto Gases").Value)/1e3
#             budget_gf = float(xr_budgets_scenario.sel(Region=reg, Time=2030, Variable="GF", Ccat="C1+C2").Value)
#             budget_pc = float(xr_budgets_scenario.sel(Region=reg, Time=2030, Variable="PC", Ccat="C1+C2").Value)
#             budget_pcc = float(xr_budgets_scenario.sel(Region=reg, Time=2030, Variable="PCC", Ccat="C1+C2").Value)
#             ndc = float(xr_total.NDC_h.sel(ISO=reg, Time=2030))
#             fig.add_trace(go.Scatter(x=[reg], y=[(base - budget_gf) / base], text="GF", textfont = {'family': "Arial black", "size": 13, "color": "black"}, marker={'symbol': "circle", "color": cols[reg_i], "size": 25, "line": {"width": 0.2, "color": 'black'}, "colorscale": "Spectral",}, mode='markers+text', showlegend=False), row=1, col=1)
#             fig.add_trace(go.Scatter(x=[reg], y=[(base - budget_pc) / base], text="PC", textfont = {'family': "Arial black", "size": 13, "color": "black"},  marker={'symbol': "square", "color": cols[reg_i], "size": 25, "line": {"width": 0.2, "color": 'black'}, "colorscale": "Spectral",}, mode='markers+text', showlegend=False), row=1, col=1)
#             fig.add_trace(go.Scatter(x=[reg], y=[(base - budget_pcc) / base], text="PCC", textfont = {'family': "Arial black", "size": 13, "color": "black"},  marker={'symbol': "diamond", "color": cols[reg_i], "size": 25, "line": {"width": 0.2, "color": 'black'}, "colorscale": "Spectral",}, mode='markers+text', showlegend=False), row=1, col=1)
#             fig.add_trace(go.Scatter(x=[reg], y=[(base - ndc) / base], text="NDC", textfont = {'family': "Arial black", "size": 13, "color": "black"},  marker={'symbol': "triangle-up", "color": cols[reg_i], "size": 25, "line": {"width": 0.2, "color": 'black'}, "colorscale": "Spectral",}, mode='markers+text', showlegend=False), row=1, col=1)

# fig.update_layout(height=1000, width = 1700)
# fig.show()

In [559]:
fig.write_image("X:/user/dekkerm/Projects/ECEMF_T5.2/Figures/Figure_costopt/Figure_costopt.svg")
fig.write_image("X:/user/dekkerm/Projects/ECEMF_T5.2/Figures/Figure_costopt/Figure_costopt.png", scale=3)

import os
from datetime import date
try:
    os.remove('X:/user/dekkerm/Projects/ECEMF_T5.2/Figures/Figure_costopt/Figure_costopt.html')
except:
    3
def html_w(typ):
    return '<html> '+typ+' <p style="font-family: Arial">'

today = date.today()
td = today.strftime("%B %d, %Y")

with open('X:/user/dekkerm/Projects/ECEMF_T5.2/Figures/Figure_costopt/Figure_costopt.html', 'a') as f:
    #f.write(html_w('<h1>')+'Figure 1 - Electricity Generation</p></h1>')
    f.write(html_w('<body>')+'Version: '+td+'</p></body>')
    f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))